In [ ]:
//export
//default_exp ModB

### Import the module we defined in ToImport.ipynb renamed to -> ModA.sc

In [1]:
//export
import $file.^.nbdev.ModA, ModA._

import $file.$           , ModA._

### Call the load_ivy script.
Located in the source folder, the load_ivy.sc script will use ammonite to download snapshots of the chisel libraries, to be cached on your system. This is necessary for EVERY chisel notebook so we can then import our chisel libraries like we would in an SBT project structure.

In [2]:
//export
import $file.^.source.load_ivy

import $file.$                

### Prevent namespace error
- To prevent an ambiguous reference to 'Input' by chisel3.Input and _root_.almond.input.Input, we have to make sure that we rename our chisel3 Input on import. It suffices to do as follows without requiring to change the name to something like IInput:
```scala
import chisel3.{Input => Input} 
```

In [3]:
//export
import chisel3.{Input => Input}
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import chisel3.tester.RawTester.test

import chisel3.{Input => Input}

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3.tester.RawTester.test

### Importing chisel modules from .sc files
To import the scripts contents as a single object do:
```scala
import $file.filename
filename.Blah()
```

To import all of the scripts contents do:
```scala
import $file.filename, filename._
Blah()
```


We can now try and test our module to see if we imported it correctly:

In [4]:
test(new Add) { c =>
    c.io.a.poke(1.U)    
    c.io.b.poke(2.U)    
    c.clock.step(1)    
    c.io.out.expect(3.U)
    
    c.io.a.poke(5.U)    
    c.io.b.poke(2.U)    
    c.clock.step(1)    
    c.io.out.expect(7.U)
}

[info] [0.001] Elaborating design...
[info] [1.297] Done elaborating.
file loaded in 0.035916667 seconds, 7 symbols, 3 statements
test Add Success: 0 tests passed in 4 cycles in 0.051422 seconds 77.79 Hz


Lets use this module to build another module:

In [5]:
//export
class ComposedModule extends Module {
    val io = IO(new Bundle {
        val a = Input(UInt(4.W))
        val b = Input(UInt(4.W))
        val c = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    val addr = Module(new Add)
    addr.io.a := io.a
    addr.io.b := io.b
    io.out := addr.io.out + io.c
}

defined class ComposedModule

In [6]:
test(new ComposedModule) { c =>
    c.io.a.poke(1.U)    
    c.io.b.poke(2.U)    
    c.io.c.poke(3.U)    
    c.clock.step(1)    
    c.io.out.expect(6.U)
    
    c.io.a.poke(5.U)    
    c.io.b.poke(2.U)    
    c.io.c.poke(7.U)    
    c.clock.step(1)    
    c.io.out.expect(14.U)
}

[info] [0.001] Elaborating design...
[info] [0.180] Done elaborating.
file loaded in 0.007460458 seconds, 13 symbols, 7 statements
test ComposedModule Success: 0 tests passed in 4 cycles in 0.014604 seconds 273.90 Hz
